In [ ]:
!pip install pyarrow

In [ ]:
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as sns
import datetime as timedelta

In [ ]:
df=pd.read_parquet(r"C:\Amex_Round2\data\amex_offers_data.parquet")
df.head()

In [ ]:
print("Shape of the dataset: ", df.shape)
 
print(df.info())

print(df.isnull().sum())

for col in df.columns:
    print(f"{col}: {df[col].nunique()} unique values")

Removal of Duplicates

In [ ]:

#Sort by customer_id, offer_id, and event_ts 
df['event_ts']=pd.to_datetime(df['event_ts'], errors='coerce')

#Keep the latest record
df=df.sort_values(['customer_id', 'offer_id', 'event_ts'], ascending=[True, True, False])
df=df.drop_duplicates(subset=['customer_id', 'offer_id'], keep='first')


Data Type Correction

In [ ]:
#Convert event_dt to datetime (MM pehle)
df['event_dt'] = pd.to_datetime(df['event_dt'], errors='coerce', dayfirst=True)

#Convert to float
numerical_columns = [col for col in df.columns if col.startswith('var_')]

for col in numerical_columns:
    df[col]=pd.to_numeric(df[col], errors='coerce')

print(df.dtypes)

Removing out of bound values

In [ ]:
#probability bw 0 and 1
probability_columns = ['var_41', 'var_42']

for col in probability_columns:
    df=df[(df[col] >= 0) & (df[col] <= 1)]


In [ ]:
#boolean columns
bool_columns = ['offer_action', 'var_44', 'var_45', 'var_46', 'var_47', 'var_48', 'var_49', 'var_50']

#convert true false to 1 and 0 respectively
for col in bool_columns:
    df[col]=df[col].replace({True: 1, False: 0})


In [ ]:
#rows with missing values deleted
df = df.dropna(subset=['customer_id', 'offer_id', 'event_ts', 'event_dt'])

#index reset
df = df.reset_index(drop=True)

In [ ]:
df.to_csv(r'../data/cleaned_amex_data.csv', index=False)

